In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input

from tensorflow.python.client import device_lib

In [ ]:
print(tf.__version__)

device_lib.list_local_devices()

2.8.0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10960389087982028575
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11320098816
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12934477382601211736
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
 xla_global_id: 416903419]

In [ ]:
seed = 22222

In [ ]:
TRAIN_DATA_DIR = "/content/drive/MyDrive/고양이_리뉴얼/train/train/"
VALIDATION_DATA_DIR = "/content/drive/MyDrive/고양이_리뉴얼/val/val/"
NUM_CLASSES = 37
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

TRAIN_SAMPLES = 6650
VALIDATION_SAMPLES = 703

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size = (IMG_WIDTH, IMG_HEIGHT),
    batch_size = BATCH_SIZE,
    shuffle = True,
    seed = seed,
    class_mode = 'categorical'
)

validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size = (IMG_WIDTH, IMG_HEIGHT),
    batch_size = BATCH_SIZE,
    shuffle = True,
    seed = seed,
    class_mode = 'categorical'
)

Found 6650 images belonging to 37 classes.
Found 703 images belonging to 37 classes.


In [ ]:
model = MobileNet(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))

for layer in model.layers[:]:
    layer.trainable = True

input = Input(shape = (224, 224, 3))
x = model(input)
x = GlobalAveragePooling2D()(x)
x = Dense(NUM_CLASSES, activation='softmax')(x)

model_similarity_optimized = Model(inputs = input, outputs = x)
model = Model(model_similarity_optimized.input, model_similarity_optimized.layers[-2].output)
model.save("/content/drive/MyDrive/고양이_리뉴얼/model_epoch_0.h5")

In [ ]:
for epoch_cnt in range(1, 11):
  print(f"epoch : {epoch_cnt}")

  model_similarity_optimized.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(lr=0.001),
              metrics=['acc'])
  model_similarity_optimized.fit_generator(train_generator,
                    steps_per_epoch = TRAIN_SAMPLES // BATCH_SIZE,
                    epochs = 1,
                    validation_data = validation_generator,
                    validation_steps = VALIDATION_SAMPLES // BATCH_SIZE)

  # model_similarity_optimized.layers.pop()
  model = Model(model_similarity_optimized.input, model_similarity_optimized.layers[-2].output)
  model.save(f"/content/drive/MyDrive/고양이_리뉴얼/model_epoch_{epoch_cnt}.h5")

epoch : 1


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


103/103 [==============================] - 78s 694ms/step - loss: 0.8594 - acc: 0.7410 - val_loss: 3.0998 - val_acc: 0.3547
epoch : 2
103/103 [==============================] - 74s 683ms/step - loss: 0.4243 - acc: 0.8633 - val_loss: 2.4755 - val_acc: 0.4484
epoch : 3
103/103 [==============================] - 74s 688ms/step - loss: 0.3070 - acc: 0.8990 - val_loss: 2.1030 - val_acc: 0.5625
epoch : 4
103/103 [==============================] - 74s 687ms/step - loss: 0.2401 - acc: 0.9224 - val_loss: 3.9502 - val_acc: 0.4266
epoch : 5
103/103 [==============================] - 74s 686ms/step - loss: 0.2108 - acc: 0.9311 - val_loss: 2.2455 - val_acc: 0.5625
epoch : 6
103/103 [==============================] - 74s 686ms/step - loss: 0.1850 - acc: 0.9385 - val_loss: 1.9724 - val_acc: 0.6469
epoch : 7
103/103 [==============================] - 74s 685ms/step - loss: 0.1561 - acc: 0.9500 - val_loss: 1.6138 - val_acc: 0.7063
epoch : 8
103/103 [==============================] - 74s 685ms/step - lo